In [1]:
import tensorflow as tf
import keras
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import sklearn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from numpy import array, argmax
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')


tfk = tf.layers
tfkl = tf.layers
print("Using TensorFlow version", tf.__version__)

seed = 420
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
print("RNG seed: ",seed)

Using TensorFlow backend.


('Using TensorFlow version', '1.8.0')
('RNG seed: ', 420)


In [ ]:
pathName = '/home/npasini1/Desktop/Recordings/new'
numFiles = []

timestep = 30
stride = 1
n_features = 33
n_gestures = 4
dataset_tot = np.empty((0,timestep,n_features))
dataset_gesture_array_tot = np.empty((0))

fileNames = os.listdir(pathName)
for fileNames in fileNames:
    if fileNames.endswith(".csv"):
        numFiles.append(fileNames)
        path = os.path.join(pathName, fileNames)
        print('Now reading file: ', path)
        dataframe = pd.read_csv(path)

        #encoding
        gesture = dataframe['Predicted Gesture']
        gesture_array = array(gesture)

        df = dataframe.drop(['Predicted Gesture','Timestamps'], axis=1)
        df = pd.DataFrame(df)

        samples = df.shape[0]
        # n_features = df.shape[1]

        dataset = np.empty((0,timestep,n_features))
        df_copy = df

        for idx in range(samples-timestep):
            for j in range(timestep):
                df_copy.iloc[idx+j,[0,1,2,7,8,9]] = df.iloc[idx+j, [0,1,2,7,8,9]] - df.iloc[idx, [0,1,2,7,8,9]]
            dataset = np.append(dataset, np.expand_dims(df_copy[idx:idx+timestep], axis=0),axis=0)
            j=0
        
        dataset_gesture_array = gesture_array[timestep:]
        dataset_tot = np.append(dataset_tot,dataset, axis=0)
        dataset_gesture_array_tot = np.append(dataset_gesture_array_tot, dataset_gesture_array, axis=0)
        print(dataset.shape)
        print(dataset_tot.shape)
        print(dataset_gesture_array.shape)
        print(dataset_gesture_array_tot.shape)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(dataset_gesture_array_tot)
print(integer_encoded)

encoded = to_categorical(integer_encoded)
print(encoded)
            
        

In [5]:
timestep = 30
stride = 1
n_features = 33
n_gestures = 4

In [2]:
# np.save("/home/npasini1/Desktop/Datasets/dataset_Nicolo_1to16.npy", dataset_tot)
# np.save("/home/npasini1/Desktop/Datasets/labels_Nicolo_1to16.npy", encoded)
dataset_tot = np.load("/home/npasini1/Desktop/Datasets/dataset_Nicolo_1to16.npy")
encoded = np.load("/home/npasini1/Desktop/Datasets/labels_Nicolo_1to16.npy")


In [3]:
X_train, X_test, y_train, y_test = train_test_split(dataset_tot, encoded, test_size=0.10, shuffle = True)
print("Train shape: ", X_train.shape)
print("Test shape: ", X_test.shape)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
print("Input shape: ", input_shape)
print("Output shape: ", output_shape)

('Train shape: ', (37220, 30, 33))
('Test shape: ', (4136, 30, 33))
('Input shape: ', (30, 33))
('Output shape: ', (4,))


In [ ]:
# dataframe = pd.read_csv('/home/npasini1/Desktop/Recordings/nic_suture_10.csv')
dataframe = pd.read_csv('/home/npasini1/Desktop/Recordings/new/Nicolo_1.csv')
dataframe.columns

In [ ]:
# one hot encoding of Gestures

gesture = dataframe['Predicted Gesture']
values = array(gesture)
print(values)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

encoded = to_categorical(integer_encoded)
print(encoded)

In [ ]:
print(values)

In [ ]:
print(integer_encoded[793])
print(values[793])

In [ ]:
# Data normalization

labels = encoded
df = dataframe.drop(['Predicted Gesture','Timestamps'], axis=1)

# min_max_scaler = preprocessing.MinMaxScaler()
# df_scaled = min_max_scaler.fit_transform(df)
# df_scaled = pd.DataFrame(df_scaled)
# df_scaled.head

df = pd.DataFrame(df)

In [ ]:
print(df.shape)

In [ ]:
samples = df.shape[0]
n_features = df.shape[1]
timestep = 30
stride = 1

dataset = np.empty((0,timestep,n_features))
df_copy = df
# np_scaled = np.array(df_scaled)
# print(dataset.shape)

# CON PREPROCESSING DEI DATI: MINMAX SCALER

# for idx in range(samples-timestep):
#     dataset = np.append(dataset, np.expand_dims(df_scaled[idx:idx+timestep], axis=0),axis=0)
#     # print(idx)

# SENZA PREPROCESSING DEI DATI


for idx in range(samples-timestep):
    for j in range(timestep):
        df_copy.iloc[idx+j,[0,1,2,7,8,9]] = df.iloc[idx+j, [0,1,2,7,8,9]] - df.iloc[idx, [0,1,2,7,8,9]]
    dataset = np.append(dataset, np.expand_dims(df_copy[idx:idx+timestep], axis=0),axis=0)
    j=0

print(dataset.shape)
dataset_labels = labels[timestep:]


In [ ]:
dataset_labels = labels[timestep:]
print(dataset_labels.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_labels, test_size=0.10, shuffle = True)
print("Train shape: ", X_train.shape)
print("Test shape: ", X_test.shape)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
print("Input shape: ", input_shape)
print("Output shape: ", output_shape)

In [16]:
import tensorflow as tf
import keras as tfk
import keras.layers as tfkl
import scipy.io
import os, glob, sys, pickle, time, math, gc
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model, load_model
from keras import optimizers
from keras.utils import plot_model
from keras import layers #Dense, LSTM, RepeatVector, TimeDistributed, Dropout, Masking, BatchNormalization, Flatten, Input, Conv2D, MaxPooling1D, Conv1D, Reshape, GRU
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, CSVLogger
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from keras import backend as K
from datetime import datetime

n_gestures = 4

def buildModelv2(timesteps, n_features):
    """
    An lstm-encoder model followed by dense layers, similar performance to just lstm
    """
    # lstm_hidden1 = 128 #*4
    # lstm_hidden2 = 64
    # dense_hidden1 = 64
    # output_layer = 15

    model_input = layers.Input(shape=input_shape)
    lstm_output = layers.LSTM(128, input_shape=input_shape, kernel_regularizer=keras.regularizers.l1(l=0.001), return_sequences=True)(model_input) #previously 96
    dropout_output = layers.Dropout(rate=0.2)(lstm_output) #previously 0.4
    batch_norm1 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dropout_output)
    lstm_output2 = layers.LSTM(64, input_shape=input_shape, kernel_regularizer=keras.regularizers.l1(l=0.001), return_sequences=False)(batch_norm1)
    dropout_output = layers.Dropout(rate = 0.3)(lstm_output2)
    batch_norm2 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dropout_output)
    repeat_vector = layers.RepeatVector(timestep)(batch_norm2)

    flatten_output = layers.Flatten()(repeat_vector)
    batch_norm2 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(flatten_output)
    dropout_output = layers.Dropout(rate = 0.2)(batch_norm2)
    dense_output1 = layers.Dense(64, activation='relu')(dropout_output)
    batch_norm3 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dense_output1)
    dense_output2 = layers.Dense(4, activation='softmax')(batch_norm3)
    lstm_classifier = Model(inputs=model_input, outputs=dense_output2)
    
    #Compile the model
    lstm_classifier.compile(loss=tfk.losses.categorical_crossentropy, optimizer=tfk.optimizers.Adam(0.5e-2), metrics=[keras.metrics.categorical_accuracy])

    return lstm_classifier

def buildModelv1(timesteps, n_features):
    """
    Simple model which yields a high accuracy of 90% on training data but sucks on the validation data
    """
    model_input = layers.Input(shape=input_shape)
    lstm_output = layers.LSTM(128, input_shape=input_shape, kernel_regularizer=keras.regularizers.l1(l=0.001), return_sequences=True)(model_input)
    dropout_output = layers.Dropout(rate=0.2)(lstm_output)
    flatten_output = layers.Flatten()(dropout_output)
    dense_output1 = layers.Dense(64, activation='relu')(flatten_output)
    dropout_output2 = layers.Dropout(rate=0.2)(dense_output1)
    dense_output2 = layers.Dense(4, activation='softmax')(dropout_output2)
    lstm_classifier = Model(model_input, dense_output2)
    #lstm_classifier.summary()
    lstm_classifier.compile(loss=tfk.losses.categorical_crossentropy, optimizer=tfk.optimizers.Adam(1e-3))

    return lstm_classifier

model = buildModelv2(timestep, n_features)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 30, 33)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 128)           82944     
_________________________________________________________________
dropout_10 (Dropout)         (None, 30, 128)           0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 128)           512       
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 64)                256       
__________

In [17]:
mc = keras.callbacks.ModelCheckpoint('/home/npasini1/Desktop/model_checkpoints/pyrecorder_Nicolo_1to16_allepochs_allsamples.h5',
                                        monitor='val_loss',
                                        mode = 'min',
                                        save_best_only=True,
                                        verbose=1)

In [18]:
batch_size = 256

history = model.fit(
    x = dataset_tot,
    y = encoded,
    batch_size = batch_size,
    epochs = 1000,
    validation_split=.10,
    #validation_steps=10,
    shuffle=False,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=200),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=45, factor=0.5, min_lr=1e-5),
        mc
    ]
).history

Train on 37220 samples, validate on 4136 samples
Epoch 1/1000
37220/37220 [==============================] - 53s - loss: 1.9983 - categorical_accuracy: 0.7756 - val_loss: 0.7939 - val_categorical_accuracy: 0.9594
Epoch 2/1000
37220/37220 [==============================] - 46s - loss: 0.9538 - categorical_accuracy: 0.8344 - val_loss: 0.6344 - val_categorical_accuracy: 0.9657
Epoch 3/1000
37220/37220 [==============================] - 43s - loss: 0.8863 - categorical_accuracy: 0.8616 - val_loss: 0.4608 - val_categorical_accuracy: 0.9659
Epoch 4/1000
37220/37220 [==============================] - 44s - loss: 0.9647 - categorical_accuracy: 0.8593 - val_loss: 0.5054 - val_categorical_accuracy: 0.9628
Epoch 5/1000
37220/37220 [==============================] - 43s - loss: 0.8921 - categorical_accuracy: 0.8631 - val_loss: 0.6656 - val_categorical_accuracy: 0.9487
Epoch 6/1000
37220/37220 [==============================] - 45s - loss: 0.8525 - categorical_accuracy: 0.8695 - val_loss: 0.5528 - 

In [9]:
model.save('/home/npasini1/Desktop/model_checkpoints/modelsave.h5')

In [10]:
model.evaluate(X_test, y_test)

4064/4136 [============================>.] - ETA: 0s

[0.058333030173183927, 0.9929883945841392]

In [ ]:
model = keras.models.load_model('/home/npasini1/nicolo_ws/src/pyrecorder/src/bestmodel_v2_senzapreprocessing.h5')

In [ ]:
a = model.predict(X_test)
print(X_test.shape)
prediction = np.argmax(a, axis=1)
print(prediction)